In [1]:
import logging
import lmdb
from gensim.models import Word2Vec
from doc_pb2 import Document, Dictionary

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
%%time
dictionary = Dictionary()
with open ('dumps/full_dict.dump', 'rb') as f:
    dictionary.ParseFromString(f.read())

Wall time: 4min 6s


In [3]:
%%time
model = Word2Vec.load('dumps/news.model ')

2017-09-12 22:52:02,509 : INFO : loading Word2Vec object from dumps/news.model 
2017-09-12 22:52:03,391 : INFO : loading wv recursively from dumps/news.model .wv.* with mmap=None
2017-09-12 22:52:03,392 : INFO : loading syn0 from dumps/news.model .wv.syn0.npy with mmap=None
2017-09-12 22:52:05,225 : INFO : setting ignored attribute syn0norm to None
2017-09-12 22:52:05,226 : INFO : loading syn1neg from dumps/news.model .syn1neg.npy with mmap=None
2017-09-12 22:52:07,984 : INFO : setting ignored attribute cum_table to None
2017-09-12 22:52:07,985 : INFO : loaded dumps/news.model 


Wall time: 6.7 s


In [4]:
%%time
db = lmdb.open('../../data/news data/documents')

url_title = {}

with db.begin() as cur:
    docs = []
    for k, v in cur.cursor():
        docu = Document()
        docu.ParseFromString(v)
        
        url_title[docu.url] = docu.title
        del docu
        if len(url_title) ==100000:
            break

db.close()

Wall time: 11.5 s


In [65]:
def find_doc_from_verb(dictionary, word):
    word2topic = dictionary.word2topic.get_or_create(word)
    
    print('word: {0}'.format(word))
    topics_with_proba = [ topic for rank, topic in word2topic.topics.items()][:10]
    
    docs = []
    
    for topic in topics_with_proba:
        topic2doc = dictionary.topic2doc.get_or_create(topic.topic_code)
        docs_with_proba = [ doc for rank, doc in topic2doc.docs.items()][:100]
        for doc in docs_with_proba:
            doc.proba = doc.proba * topic.proba
            docs.append(doc)
    
    docs.sort(key=lambda doc: doc.proba, reverse=True)
    
    largest = docs

    s = set()
    hundred_largest = []
    for n in largest:
        if n.url not in s:
            s.add(n.url)
            hundred_largest.append(n.url)
        if len(hundred_largest) == 101:
            break
    with db.begin() as txn:
        i = 0
        lines = set()
        for url in hundred_largest:
            i += 1
            print('\nNo. {0}'.format(i))
            document = Document()
        #           print(doc)
            s = txn.get(url.replace('\n', '').encode('utf-8'))
        #           print(type(s))
            if s is None:
                print('none')
            document.ParseFromString(s)
            line = ''
            if document.title != '':
                line = document.title
            else:
                line = document.content
            if line not in lines:
                lines.add(line)
                print(line)
#         print('\n')
#     return hundred_largest

In [78]:
hundred_largest = find_doc_from_verb(dictionary, '奥运会')

word: 奥运会

No. 1
第十一届亚运会男子花剑个人决赛后，中国选手叶冲被教练和队友举起。

No. 2
今天举重赛场将产生两枚金牌，中国选手目标是保二争一。中国名将吴景彪表示：“我的任务就是捍卫中国男举小级别荣誉！”５６公斤级是中国男举传统优势项目，四年前在北京，中国选手龙清泉表现出色，勇夺金牌，之后四年中国队在该级别一直孤独求败。此次出征的吴景彪连夺两届世锦赛总成绩冠军，实力同样了得。吴景彪表示，“举重是个人项目，只要把过程准备好，该拿的肯定跑不掉。”女子５３公斤级，国内奥运选拔赛冠军纪静没有来到伦敦奥运会，出现在伦敦的是让外界感到意外的周俊，她职业生涯不多的亮点就是今年举重亚锦赛获得的一枚铜牌。这个项目金牌之争主要集中在中哈土三国选手之间，哈萨克斯坦的劲敌就是祖尔菲亚，土耳其的达斯德伦则是容易被忽视的一股力量。

No. 3
２００８年北京奥运会上，薛晨／张希曾与另一对女沙组合田佳／王洁，分别夺得一铜一银。北京奥运会后，田佳／王洁宣布退役，薛晨／张希在世界赛场的征战中逐渐跻身一流好手的行列。伦敦奥运会，她俩能否更进一步，令人期待。进入２０１２赛季，薛晨／张希在世界沙滩排球大满贯赛中取得过相当出色的战绩，共获得３站冠军、一次亚军、一次季军。由于伤病困扰，她们也有成绩不理想的时候。因此，尽管薛晨／张希目前高居世界排名第二，国家体育总局排球运动管理中心主任徐利坦言，伦敦奥运会共有２４对选手参赛，无论是小组赛还是淘汰赛，每一场都不能有一点闪失，十六进八、八进四这两轮淘汰赛对薛晨／张希非常关键。徐利说，伦敦奥运会上，朱莉安娜／拉里萨和世界排名第三的上届奥运会冠军美国组合沃尔什／梅·特雷纳将是薛晨／张希最强劲的对手。虽然对手的年龄都偏大、体能上会有些欠缺，但她们的大赛经验都非常丰富，薛晨／张希在比赛中要做好打持久战的心理准备，才有取胜的希望。

No. 4
男团夺冠国球乒乓夺金率高达１００％，一举囊括男女团体、单打四枚金牌，甚至在全部比赛中保持一盘不败的惊人奇迹。

No. 5
女子举重４８公斤级决赛，中国选手王明娟以总成绩２０５公斤如愿夺得金牌，这也是中国代表团在这次奥运会上获得的第二枚金牌。

No. 6
４年前的北京奥运会上，北京体育健儿在主场收获４．５枚金牌，以及２银４铜。在本次参加伦敦奥运会的中国代表团中，有２２名在市体育局注册的运动员，其中

In [8]:
db = lmdb.open('../../data/news data/documents')

In [51]:
# doc = hundred_largest[0]
doc = 'http://cpc.people.com.cn/GB/87228/18164397.html\n'



No. 1
老狼受访时称与石康是一个学校的，是走读的大学，不给学生提供宿舍，但是他们两个在校外住在一起，是特别好的哥儿们。

No. 2
同学们，盖楼过百，发大图哦！

No. 3
山西大学的校长和他的副手。

No. 4
深受皇民化教育的日本学校女生组建的“慰安妇团”

No. 5
欣赏老师的佳作。向老师学习！

No. 6
华侨大学首任校长廖承志与华大学生在一起华侨大学首任校长廖承志与华大学生在一起。

No. 7
今年合肥市首次将五年制高职、中专学校志愿和普通高中志愿一起填报，如何填报中职志愿也成了一部分考生和家长关注的焦点。记者了解到，五年制高职及普通中专录取同样按志愿顺序进行投档，以考生文化课成绩从高分到低分进行录取，第一志愿录满的学校，不再录取第二志愿考生，每个考生只能被录取一次。根据合肥市区中招日程安排，考生将于６月２９日～７月８日填报中招志愿，高中阶段招生共设置７个批次１５个志愿，其中第五批次为五年制高职院校招生。初中起点五年制高职也就是通常说的“３＋２”培养模式，三年完成中专课程，后两年升入对应的高职院校读专科，五年后毕业取得的学历是国家认可的大专学历证书。五年制高职、普通中等专业学校录取由合肥市教育考试院负责组织实施。对于符合报考条件的考生，考试院将按照招生志愿顺序和计划数，以招生学校为单位分批次择优录取，不设平行志愿。招生实行计算机投档、录取，按志愿顺序进行投档，以考生文化课成绩从高分到低分进行录取。第一志愿录满的学校，不再录取第二志愿考生。合肥市教育考试院也提醒考生，高中阶段招生志愿表应由考生本人填写，家长签名确认，任何学校和个人不得要求学生违背意愿填报志愿。（吴曦、胡霈霖）

No. 8
美智子让孩子享受普通孩子的成长环境。

No. 9
美智子让孩子享受普通孩子的成长环境。

No. 10
美智子让孩子享受普通孩子的成长环境。

No. 11
美智子让孩子享受普通孩子的成长环境。

No. 12
恢复高考后，北京大学第一批新生、物理系一年级的学生在老师辅导下做基础物理实验。

No. 13
今年合肥市首次将五年制高职、中专学校志愿和普通高中志愿一起填报，如何填报中职志愿也成了一部分考生和家长关注的焦点。记者了解到，五年制高职及普通中专录取同样按志愿顺序进行投档，以考生文化课成绩从高分到低分进行录取，第一志愿录满的学校，不再录取第

In [43]:
document = Document()
document.ParseFromString(s)
document.content

'入学即送笔记本电脑？对，是真的，这是江西城市职业学院开出的招生条件。\ue40c昨日，由省考试院主办的全国在皖招生三本院校及高职（专科）专场咨询会在安徽新华学院举办，２００多所院校现场揽生源，高校展台前“冰火两重天”。\ue40c记者注意到，大部分考生还是倾向於咨询安徽本地高校及沿海发达地区的一些高校，一些地理位置欠佳的高校展台前，咨询的考生就少很多。\ue40c为了招到足够的生源，一些院校甚至开出了诱人的条件：民办院校江西城市职业学院就宣称，入学即送笔记本，还为考生提供专升本辅导平台……该校今年在我省招收４２０人，但有可能招不满。\ue40c还有学校打出了专科层次家长和考生们最为关心的“就业”牌。天津冶金职业技术学院表示，入学第二年学生便可以与企业进行互选，然后实行订单式培养，实现“百分百”就业。\ue40c记者了解到，今年，考生仍比较关注医学、土建等热门专业。对此，安徽三联学院招生办负责人建议，适合自己的专业才是最好的。他还提醒考生，在三本征集志愿和专科志愿的选择上，达到条件的考生可以填报三本征集志愿，不用担心时间冲突。\ue40c对於那些分数稍高的考生，安徽大学江淮学院的招生办负责人也提醒考生，填报志愿时不要以为分数高就不填服从调剂，往年有些考生因此而没有被录取。（实习生\u3000郝想想\u3000记者\u3000武静）'

In [12]:
with db.begin() as txn:
    print(txn.get(doc.replace('\n', '').encode('utf-8')))

b'\n!0d121d977a7e1331-bb9f32f06cef7000\x12<http://comic.people.com.cn/n/2012/0716/c144727-18528097.html"\xd0\x02\xe8\xbf\x91\xe5\xb9\xb4\xe6\x9d\xa5\xe7\x9a\x84\xe4\xb8\xad\xe8\x80\x83\xe5\x89\x8d\xe5\x90\x8e\xef\xbc\x8c\xe6\xb2\xb3\xe5\x8c\x97\xe4\xb8\x80\xe4\xba\x9b\xe9\xab\x98\xe4\xb8\xad\xe5\x9c\xa8\xe5\x90\x84\xe5\x9c\xb0\xe8\xae\xbe\xe7\x82\xb9\xef\xbc\x8c\xe8\x81\x98\xe7\x94\xa8\xe6\x8b\x9b\xe7\x94\x9f\xe4\xbb\xa3\xe8\xa1\xa8\xe8\xb7\xa8\xe5\x9c\xb0\xe6\x8b\x9b\xe7\x94\x9f\xef\xbc\x9b\xe7\x94\x9a\xe8\x87\xb3\xe7\xbb\x99\xe9\x92\xb1\xe3\x80\x81\xe7\xbb\x99\xe7\x89\xa9\xef\xbc\x8c\xe8\xb4\xbf\xe8\xb5\x82\xe4\xb8\xaa\xe5\x88\xab\xe5\x88\x9d\xe4\xb8\x89\xe8\x80\x81\xe5\xb8\x88\xef\xbc\x8c\xe4\xbb\xa5\xe6\x8b\xbf\xe5\x88\xb0\xe5\x88\x9d\xe4\xb8\xad\xe5\xb0\x96\xe5\xad\x90\xe7\x94\x9f\xe7\x9a\x84\xe4\xbf\xa1\xe6\x81\xaf\xe5\x92\x8c\xe5\x90\x8d\xe5\x8d\x95\xe3\x80\x82\xe4\xb8\x80\xe4\xbd\x8d\xe6\xb2\xb3\xe5\x8c\x97\xe7\x9c\x81\xe4\xb8\xad\xe5\xad\xa6\xe6\x95\x99\xe8\x82\xb2\xe4\xb8\x93

In [19]:
word2topic = dictionary.word2topic.get_or_create('收音机')

In [26]:
word2topic = dictionary.word2topic.get_or_create('放假\n')

In [39]:
from pprint import pprint
pprint(word2topic.topics)

{0: topic_code: 79
proba: 124.14511455524523
, 1: topic_code: 300
proba: 40.71874856649096
, 2: topic_code: 531
proba: 26.84058659150057
, 3: topic_code: 535
proba: 0.001000001534162212
, 4: topic_code: 156
proba: 0.0010000001241843982
, 5: topic_code: 840
proba: 0.0010000000789769654
, 6: topic_code: 740
proba: 0.0010000000510957355
, 7: topic_code: 969
proba: 0.0010000000168033808
, 8: topic_code: 430
proba: 0.0010000000109078536
, 9: topic_code: 939
proba: 0.0010000000086586045
, 10: topic_code: 886
proba: 0.0010000000076688398
, 11: topic_code: 289
proba: 0.0010000000035065523
, 12: topic_code: 802
proba: 0.0010000000033654364
, 13: topic_code: 372
proba: 0.0010000000031550609
, 14: topic_code: 190
proba: 0.0010000000022989458
, 15: topic_code: 475
proba: 0.0010000000020446251
, 16: topic_code: 128
proba: 0.0010000000017828582
, 17: topic_code: 490
proba: 0.001000000001626221
, 18: topic_code: 649
proba: 0.0010000000010682627
, 19: topic_code: 654
proba: 0.001000000001063047
, 20: 

b'sadf'